In [1]:
from surprise_cf import *
from cf_utils import *

In [2]:
masked_R_coo, unmasked_R_coo, keep_item_idxs = get_data_from_dataloader()
mask_coo = sparse.coo_matrix(logical_xor(masked_R_coo, unmasked_R_coo))

nnzs = masked_R_coo.getnnz(axis=1)
warm_users = nnzs > 2

loading the data...downloaded in 7 seconds


In [3]:
# generated_users_file = '/mnt/nfs/scratch1/neerajsharma/model_params/generated_1000_user_neighbors_more_sparse.npy'
# generated_items_file = '/mnt/nfs/scratch1/neerajsharma/model_params/generated_1000_item_neighbors_more_sparse.npy'
# generated_users = np.load(generated_users_file, allow_pickle=True).item()
# generated_items = np.load(generated_items_file, allow_pickle=True).item()
# for key, value in generated_users.items():
#     generated_users[key] = value[:,keep_item_idxs]
# num_user_ids = len(generated_users.keys())
# num_item_ids = len(generated_items.keys())
# user_neighbor_per_id, user_neighbor_dim = generated_users[list(generated_users.keys())[0]].shape
# item_neighbor_per_id, item_neighbor_dim = generated_items[list(generated_items.keys())[0]].shape
# num_generated_users = num_user_ids * user_neighbor_per_id
# num_generated_items = num_item_ids * item_neighbor_per_id

# generated_users_vectors = np.array([v for v in generated_users.values()]).reshape(num_generated_users, user_neighbor_dim)
# generated_users_coo = sparse.coo_matrix(generated_users_vectors)
# false_coo = sparse.coo_matrix(np.zeros_like(generated_users_vectors, dtype=bool))
# masked_R_coo = sparse.vstack([masked_R_coo, generated_users_coo])
# unmasked_R_coo = sparse.vstack([unmasked_R_coo, generated_users_coo])
# mask_coo = sparse.vstack([mask_coo, false_coo])

# generated_items_vectors = np.array([v for v in generated_items.values()]).reshape(num_generated_items, item_neighbor_dim)
# filler = np.zeros((num_generated_items, num_generated_users))
# generated_items_vectors = np.concatenate((generated_items_vectors, filler), axis=1)
# false_coo = sparse.coo_matrix(np.zeros_like(generated_items_vectors.T, dtype=bool))
# generated_items_coo = sparse.coo_matrix(generated_items_vectors.T)

# masked_R_coo = sparse.hstack([masked_R_coo, generated_items_coo])
# unmasked_R_coo = sparse.hstack([unmasked_R_coo, generated_items_coo])
# mask_coo = sparse.hstack([mask_coo, false_coo])
# # aug = True
# generated_users, generated_items = None, None

mask_csr = mask_coo.tocsr()
unmasked_vals_csr = unmasked_R_coo.multiply(mask_coo)
unmasked_vals_coo = sparse.coo_matrix(unmasked_vals_csr)
unmasked_cold_coo = only_cold_start(masked_R_coo, unmasked_vals_coo, warm_users)
ks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15]


num users total =  62926
num cold start users =  35946


In [4]:
algo = SVD(verbose=False)
trainset, testset, cold_testset = setup(masked_R_coo, unmasked_vals_coo, unmasked_cold_coo)


make train and test sets...done in 1 seconds


In [5]:
algo.fit(trainset)

In [6]:
predictions = algo.test(testset)

In [7]:
model = Model(name='SVD', algo=SVD(verbose=False), ks=ks, ground_truth=unmasked_vals_coo, mask=mask_coo, ground_truth_cold=unmasked_cold_coo)

In [8]:
model.train(trainset)

training  SVD ... done in  11 seconds


In [17]:
model.predict(testset, cold_testset)
model.evaluate_all_users()

evaluating all users SVD ... evaluating cold users SVD ... 

In [10]:
full_prediction_matrix = np.dot(model.algo.pu, model.algo.qi.T)

In [11]:
%reload_ext autoreload
model.get_diy_predictions(trainset.global_mean)

In [18]:
mae, rmse = MAE_and_RMSE(model.full_prediction_matrix, model.ground_truth, model.mask)
print(mae, model.mae)
print(rmse, model.rmse)

0.7396750905267852 0.6855819499332642
1.0215817211995444 0.9739931308149864


In [24]:
bias_model =  Model(name='bias only', algo=BaselineOnly(verbose=False, bsl_options = {'method': 'sgd','learning_rate': .00005,}), ks=ks)
bias_model.train(trainset)
bias_model.predict(testset, cold_testset)
bias_model.evaluate_all_users()


training  bias only ... done in  1 seconds
evaluating all users bias only ... evaluating cold users bias only ... 

In [26]:
preds = np.zeros_like(model.full_prediction_matrix)
for pred in bias_model.predictions:
    u, i, v = pred[0], pred[1], pred[2]
    preds[u,i] = v

In [29]:
mae, rmse = MAE_and_RMSE(preds, model.ground_truth, model.mask)
print(mae, bias_model.mae)
print(rmse, bias_model.rmse)

0.0 0.757013384004379
0.0 1.017178792304042
